# QOSF Mentorship program: Cohort 7, Screening Task 2

## Problem: Is rectangle?
In this notebook, I am going to create an algorithm that answers the following questions using quantum computers:

> **Given four positive integers A, B, C and D, is it possible to build a rectangle whose sides have lengths A, B, C and D, in any order?**

This problem is trivial to solve, because the only condition A, B, C and D must fulfill is to be equal in pairs, i. e., we must evaluate the following condition:
    
> **A = B and C = D  or  A = C and B = D  or  A = D and B = C**
    
If in addition all four numbers are equal (a square would be formed) the answer is "Yes" as well, because, strictly, a square is a [rectangle](https://en.wikipedia.org/wiki/Rectangle), a "square rectangle", as opposed to non-square or oblong rectangles.

This problem can therefore be solved evaluating which ones of the 6 following equality conditions hold:

> **A = B**

> **A = C**

> **A = D**

> **B = C**

> **B = D**

> **C = D**
    
It can be easily proved that when four or six (this latter case corresponding to the square rectangle case) of these conditions are fulfilled, a rectangle can be formed. Also, not all of these conditions need to be evaluated to check the equality in pairs of the four integers.

Here, I solved this problem checking these equality conditions using quantum computers. In particular, I will work with discrete-values gate-model quantum computers, like the superconducting quantum computers created by IBM, and write quantum circuits to be executed with them. Algorithm will be created using Qiskit library

There are a lot of ways of implementing a program that solves this problem, some of them less trivial than others. I have created two different circuits corresponding to different approaches to the solution. As a general criteria, I have tried to use the minimum possible qubits and also the minimum quantum logic gates, in order to be able to obtain the better possible performance and results when running these circuits using real quantum computers currently available. When both criteria (minimize qubits and logic gates) are not simultaneously possible, a convenient trade-off has been established.

Hint: The circuits have not been designed to be run with currently freely available quantum computers, so they will not necessary be runnable with real quantum computers. Therefore, the Aer simulator of Qiskit has been used to run the circuits.

## Checking equalities using quantum circuits

As described below, checking equality conditions is necessary to solve this problem. Therefore, a method to compare two numbers using quantum circuits is needed. I consider here three ways to evaluate equalities using quantum circuits:

### 1. Swap test
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}\newcommand{\bra}[1]{\left\langle{#1}\right|}$
Given two qubits in states $\phi$ and $\psi$, the [Swap test](https://doi.org/10.1137/S0097539796302452) computes its difference in an extra qubit using a controlled SWAP gate:

<img src="img/quantum_swap_test.png" width="400" height="250">

The probability of measuring 0 in the extra qubit is $ P(\ket{0}) = 1/2 + 1/2|\bra{\phi}\ket{\psi}|^2$. Therefore, if $\phi$ and $\psi$ qubits are in the same state, $P(\ket{0}) = 1$, and the more different $\phi$ and $\psi$ are, the closer will be $P(\ket{0})$ to $1/2$.

A circuit based on this test can be constructed to solve the is_rectangle problem. This could be done using some parametric gate to encode integers in qubits and comparing them with the Swap test. However, I considered that the use of this test is not appropriate in terms of qubits and logic gates minimization. The main reason is that states $\phi$ and $\psi$ are affected by the test because they may be swapped, and because of this, they can not be used again in the circuit. A circuit based on Swap test requires 3 qubits per comparison, which is clearly suboptimal. Let's check other alternatives.

### 2. Quantum kernel

Another way to compare two numbers with a quantum circuit is to encode one of them using a parametric unitary transformation $U$, and the other, using the adjoint parametric unitary transformation $U^\dagger$, using a single qubit for both:

<img src="img/quantum_kernel.png" width="400" height="250">

This constructions is the one commonly used to compute quantum kernels. It is straightforward to prove that, using an appropriate choice for $U$, the original state remains unchanged (and therefore we should measure $0$ with probability 1, if the qubits was initialized to $\ket{0}$) if and only if the parameters of $U$ and $U^\dagger$ are equal. Furthermore, the bigger the difference between the parameters, the bigger the probability of measuring $1$.

There are many ways to construct a circuit a circuit using this method to compare integers. For example, we could use this method to check which of the 6 equality conditions mentioned above fulfill, using 6 qubits. However, the nature of the problem, and the bidimensionality of the Bloch sphere allow us to use an alternative approach, using 3 qubits to perform comparison of integers in pairs.

This is the circuit I constructed to solve this problem using quantum kernel-like construction:

<img src="img/kernel_circuit_scheme.png" width="500" height="300">

It can be proved that, if the integers are normalized to be inside the interval $(0, \pi)$, the final result of each one of the qubits can be $0$ if and only if the couple of integers encoded with the $R_y$ gates are equal, and also the couple of integers encoded using the $R_z$ gates. Thus, this circuit checks simultaneously all the equality conditions on A, B, C, and D, doing it in pairs.

In conclusion, if one of this qubits is zero after the application of the parametric unic gates, a rectangle can be formed with A, B, C and D. Thus, if the bit string '111' is absent, the answer will we "Yes", and "No" otherwise: this is the condition we are going to check. Additionally, if all of them are zero, a square can be formed. Next is the code implementing this circuit.

In [1]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer
from qiskit.providers.aer import AerSimulator
import numpy as np
import math

In [2]:
# Function to normalize integers between 0 and pi, without including them

def normalize(integers):
    minimum = min(integers) - 1
    maximum = max(integers) + 1
    length = maximum - minimum
    return [ (num - minimum) * np.pi / length for num in integers ]

def is_rectangle_kernel(A, B, C, D, shots, threshold):
    
    integers = [A, B, C, D]
    [a, b, c, d] = normalize(integers)

    qubits = QuantumRegister(3, 'q')
    bits = ClassicalRegister(3, 'c')

    qc = QuantumCircuit(qubits, bits)

    qc.ry(a, qubits[0])
    qc.rz(b, qubits[0])
    qc.rz(-c, qubits[0])
    qc.ry(-d, qubits[0])
    
    qc.ry(a, qubits[1])
    qc.rz(b, qubits[1])
    qc.rz(-d, qubits[1])
    qc.ry(-c, qubits[1])

    qc.ry(a, qubits[2])
    qc.rz(c, qubits[2])
    qc.rz(-d, qubits[2])
    qc.ry(-b, qubits[2])

    qc.measure(qubits[0], bits[0])
    qc.measure(qubits[1], bits[1])
    qc.measure(qubits[2], bits[2])

    sim = AerSimulator()  # make new simulator object
    job = sim.run(qc, shots=shots)      # run the experiment
    result = job.result()  # get the results
    counts = result.get_counts()    # interpret the results as a "counts" dictionary
    return '111' not in counts or counts['111'] < shots * threshold

is_rectangle_kernel(11, 11, 11, 11, 100000, 0.005)

True

### 3. Binary quantum circuit

The last approach I tested is to build a binary quantum circuit to check the equality conditions on the integers. This means that the [basis encoding](https://learn.qiskit.org/course/machine-learning/data-encoding#data-3-0) technique will be used to encode the integers into qubits. To do so, we first deduce hoy many bits we need to encode the bigger integer, and encode all the integers with that bit length: let's call it $m$.

There are many ways to create a circuit and create an algorithm to compute is_rectangle function, some of them more trivial than others. For example, we could use $6m$ qubits to encode together the 6 pairs of integers and thus to check if they are equal, measuring all the qubits. However, I preferred to construct a circuit which use less qubits and, if possible, to encode the final binary result of is_rectangle in a single qubit.

Here is the circuit I created, which uses approximately $4m$ qubits ($3m$ qubits for integer pairs, $l \approx m$ ancilla qubits and 3 additional register qubits) to compute the function is_rectangle:

<img src="img/binary_circuit_scheme.jpg" width="600" height="600">

Let's explain one by one the steps this circuit implements:

- Transform integers into binary numbers of length $m$
- Encode these integers in binary format in the $3m$ first qubits, as can be seen on the image. The integer pairs AB, AC and AD are thus encoded together: let's call this a "comparison block". Obviously, the final result of a block of m qubits will be 0 for all the qubits if and only if the equality between the couple holds.
- Now, we perform a "checking procedure" to check if the comparison block qubits are all zero or not, using ancilla qubits. The process is the following:
    - Use the ancilla qubits to compute the final result of a comparison block of m qubits, i. e. of a couple of integers. To do so (not detailed in the scheme above), the qubits of each block are taken two by two, and an ancilla qubit is taken as the target. For these three qubits an X gate is applied, and after that, a Toffoli gate is applied using the ancilla qubtis as target. This construction ensures that the ancilla qubit will remain in the $\ket{0}$ state if and only if bot qubits of the comparison block are zero, and flipped otherwise. All the rest of the qubits are evaluated in this way using new ancilla qubits; if $m$ is even, the last one is directly "passed" to another ancilla qubit with a CX gate. Thus, we use $ceil(m/2)$ ancilla qubits to perform the first evaluation round for a comparison block
    - We proceed with this procedure recursively, starting now with the first ancilla qubit, and using additional ancilla qubits. In this step we need $ceil(ceil(m/2)/2)$ additional ancilla qubits.
    - Finally, we stop the procedure when only one or two ancilla qubits remain. The number of ancilla qubits depends of m on a nontrivial way, but is similar. The final result of this contraction procedure is "stored" with a Toffoli gate (two ancilla remain) or a cx (one ancilla remains) in the first register qubit
    - Finally apply the sets of Toffoli and x gates performed during the procedure in the inverse order. The prupose of this is to leave the comparison block and the ancilla qubits unaffected by the checking procedure, so they can be used again. This reversion step is optional, and is not necessary if the qubits are not needed anymore
- Once the checking procedure is performed for the AB couple, and the result is stored in the first register qubit, we do the same with AC and AD couples (with reversion) and store the result in the first register qubit. Thus, this register qubits will be 0 if all the equalities hold, or just one of them.
- After that, we apply CX gates to "pass" the state of couple AB (block 1) to AC (block 2): the result will be the same that if we would introduced CD couple in block 2. After, we do the same to pass BC state to AD (block 3), so in block 3 we have the same state we would have if we would have apply ABCD integers. We now compute again the checking procedure using ancilla qubits (without reversion) and store the result in the second register qubit. Thus, we check if the equalities in pairs hold. 
- It can be proved that, if the result of ABCD is zero, there are only two possibilities: a rectangle is possible, or all the four integers are different but comine in a way that ABCD is zero. to avoid this case, we computed all the equalities AB, AC and AD in the first register qubit, so if all of them are different to zero, this register qubits too. This is why the rectangle is only possible if the first and the second qubits are both in state $\ket{0}$. We perform a simple circuit to check this and store the result in the third register qubit. a measure of the state of this qubit will give us the result we wanted.

Now the algorithm has been explained, let's implement the code.

In [85]:
def getBinarySize(integers):
    return int(max(np.floor(np.log2(integers)) + 1))

def getAncillaSize(binarySize):
    nextRound = binarySize
    totalAncillas = 0
    while nextRound > 2:
        nextRound = math.ceil(nextRound/2)
        totalAncillas += nextRound
    return totalAncillas
    
def toBinaryList(x, n):
    return list(map(int, list(str(bin(x)[2:]).zfill(n))))

def ancillas_register(qc, qubits, initial_qubits, qubit_position, ancilla_position, registry, rollback):
    rollback_register = []
    while initial_qubits > 2:
        final_qubits = math.floor(initial_qubits / 2)
        for i in range(0, final_qubits):
            qc.x(qubits[2*i + qubit_position])
            qc.x(qubits[2*i + qubit_position + 1])
            qc.x(qubits[i + ancilla_position])
            qc.ccx(qubits[2*i + qubit_position], qubits[2*i + qubit_position + 1], qubits[i + ancilla_position])
            rollback_register.append([2*i + qubit_position, 2*i + qubit_position + 1, i + ancilla_position])

        if (initial_qubits % 2) != 0:
            qc.cx(qubits[initial_qubits - 1 + qubit_position], qubits[final_qubits + ancilla_position])
            final_qubits += 1

        initial_qubits = final_qubits
        qubit_position = ancilla_position
        ancilla_position += final_qubits

    if (initial_qubits == 2):
        qc.x(qubits[qubit_position])
        qc.x(qubits[qubit_position + 1])
        qc.x(qubits[registry])
        qc.ccx(qubits[qubit_position], qubits[qubit_position + 1], qubits[registry])
        # Rollback
        if (rollback):
            qc.x(qubits[qubit_position])
            qc.x(qubits[qubit_position + 1])
    elif (initial_qubits == 1):
        qc.cx(qubits[qubit_position], qubits[registry])

    # Rollback
    if (rollback and len(rollback_register) > 0):
        for step in list(reversed(rollback_register)):
            qc.ccx(qubits[step[0]], qubits[step[1]], qubits[step[2]])
            qc.x(qubits[step[0]])
            qc.x(qubits[step[1]])
            qc.x(qubits[step[2]])
            
def combine_integers(qc, qubits, initial_qubit_1, initial_qubit_2, limit):
    for i in range(limit):
        qc.cx(qubits[i + initial_qubit_1], qubits[i + initial_qubit_2])

def get_register_result(qc, qubits, bits, registers):
    qc.x(qubits[registers[0]])
    qc.x(qubits[registers[1]])
    qc.ccx(qubits[registers[0]], qubits[registers[1]], qubits[registers[2]])
    qc.measure(qubits[registers[2]], bits[0])


def is_rectangle_circuit(A, B, C, D, shots):

    integers = [A, B, C, D]
    binarySize = getBinarySize(integers)
    integerToBits = [ toBinaryList(integer, binarySize) for integer in integers ]
    ancillaSize = getAncillaSize(binarySize)
    qubits = QuantumRegister(3 * binarySize + ancillaSize + 3, 'q')
    bits = ClassicalRegister(1, 'c')
    registers = [i + 3 * binarySize + ancillaSize for i in range(3)]
    
    qc = QuantumCircuit(qubits, bits)

    #AB
    qc.x([i for i, el in enumerate(list(range(binarySize))) if (integerToBits[0][i] == 1)])
    qc.x([i for i, el in enumerate(list(range(binarySize))) if (integerToBits[1][i] == 1)])

    #AC
    qc.x([i + binarySize for i, el in enumerate(list(range(binarySize))) if (integerToBits[0][i] == 1)])
    qc.x([i + binarySize for i, el in enumerate(list(range(binarySize))) if (integerToBits[2][i] == 1)])

    #AD
    qc.x([i + 2 * binarySize for i, el in enumerate(list(range(binarySize))) if (integerToBits[0][i] == 1)])
    qc.x([i + 2 * binarySize for i, el in enumerate(list(range(binarySize))) if (integerToBits[3][i] == 1)])

    # Register AB comparison, with rollback
    ancillas_register(qc, qubits, binarySize, 0, 3*binarySize, registers[0], True)

    # Register AC comparison, with rollback
    ancillas_register(qc, qubits, binarySize, binarySize, 3*binarySize, registers[0], True)

    # Register AD comparison, with rollback
    ancillas_register(qc, qubits, binarySize, 2 * binarySize, 3*binarySize, registers[0], True)

    # Combine AB with AC and overwrite in AC. Result: BC
    combine_integers(qc, qubits, 0, binarySize, binarySize)

    # Combine BC with AD and overwrite in AD. Result: ABCD
    combine_integers(qc, qubits, binarySize, 2 * binarySize, binarySize)

    # Register ABCD comparison, without rollback
    ancillas_register(qc, qubits, binarySize, 2 * binarySize, 3*binarySize, registers[1], True)

    # Final circuit to get the result: 1 means True, 0 means False
    get_register_result(qc, qubits, bits, registers)
    
    sim = AerSimulator()  # make new simulator object
    job = sim.run(qc, shots=shots)      # run the experiment
    result = job.result()  # get the results
    counts = result.get_counts()    # interpret the results as a "counts" dictionary
    false_shots = counts['0'] if '0' in counts else 0
    true_shots = counts['1'] if '1' in counts else 0
    return true_shots > false_shots

is_rectangle_circuit(8, 11, 11, 8, 100000)

True

In [86]:
#Tests
print(is_rectangle_kernel(11, 11, 11, 11, 100000, 0.005))
print(is_rectangle_circuit(11, 11, 11, 11, 100000))

print(is_rectangle_kernel(11, 9, 11, 11, 100000, 0.005))
print(is_rectangle_circuit(11, 11, 11, 9, 100000))

print(is_rectangle_kernel(7, 11, 11, 7, 100000, 0.005))
print(is_rectangle_circuit(7, 7, 11, 11, 100000))

print(is_rectangle_kernel(8, 11, 8, 11, 100000, 0.005))
print(is_rectangle_circuit(8, 11, 8, 11, 100000))

print(is_rectangle_kernel(2, 2, 11, 11, 100000, 0.005))
print(is_rectangle_circuit(2, 11, 11, 2, 100000))

print(is_rectangle_kernel(3, 5, 8, 2, 100000, 0.005))
print(is_rectangle_circuit(4, 5, 9, 18, 100000))

print(is_rectangle_kernel(76, 42, 76, 42, 100000, 0.005))
print(is_rectangle_circuit(126, 126, 215, 215, 100000))

True
True
False
False
True
True
True
True
True
True
False
False
True
True


# Conclusion

The two approaches used to solve the problem have advantages and disadvantages. Let's analyze some of them:

- The kernel circuit uses only 3 qubits regardless the size of the integer. However, the binary circuit needs much more qubits, and this amount grows logarithmically with the size of N. This means that the inary circuit can be different to be used in NISQ era devices.
- The kernel circuit uses only 4 gates at each qubit, while the binary circuit is much deeper. This is also a disavantage with respect to applicability in NISQ era devices
- Finally, the kernel circuit is extremely simple, while the binary circuit is highly convoluted.
- However, the kernel circuit has an enormous disadvantage. The problem with it is that there is a situation where this circuit possibly fails: when the difference between the ninimum and maximum integers is very big (i. e., rotations are extremely close to 0 and $\pi$), and when the difference between the integers is very low compared to the maximum-minimum difference (rotations are extremely similar). In this case, a NISQ era device will likely fail. This is because this algorithm is not resilient to failure, because it is based in the absence of a state ('111') which may be measured in a significant proportion if the quantum computer is not fault-tolerant. The binary circuit is designed to not to have this problem: its result can be only 0 or 1 at each step, and only a huge error propagation (due to a lot of qubits and gates for huge integers) may cause this algorithm fail.

My final conclusion is that no one of these approaches is better than the other. If we do not take into account the triviality of the problem (if we "solve" it classically), and we analyze these circuits like they would be going to be used for practical purposes, both algorithms have problems in the NISQ era and, in fact, both will require fault-tolerant quantum computing to work with huge integers.